Change stats to per game stats

In [13]:
import pandas as pd
from sklearn import linear_model

In [14]:
categories = ['G', 'A', 'PN', 'SH', 'FO_W', 'HIT']

In [15]:
def get_stats(stat):
    # import stats
    season12 = pd.read_csv('Stats/war-on-ice-2012.csv')
    season13 = pd.read_csv('Stats/war-on-ice-2013.csv')
    season14 = pd.read_csv('Stats/war-on-ice-2014.csv')
    # set name as index
    season12.index = season12.Name
    season13.index = season13.Name
    season14.index = season14.Name
    # drop columns
    season12 = season12.drop(['Unnamed: 0', 'Team', 'Salary', 'AAV', 'season', 'Name', 'pos'], 1)
    season13 = season13.drop(['Unnamed: 0', 'Team', 'Salary', 'AAV', 'season', 'Name', 'pos'], 1)
    season14 = season14.drop(['Unnamed: 0', 'Team', 'Salary', 'AAV', 'season', 'Name', 'pos'], 1)
    # drop retired players
    season12 = season12[season12.index.isin(season14.index)]
    season13 = season13[season13.index.isin(season14.index)]
    # add year to column names
    season12.rename(columns=lambda x: x + '_12', inplace=True)
    season13.rename(columns=lambda x: x + '_13', inplace=True)
    # merge dataframes
    temp = season12.merge(season13, left_index = True, right_index = True)
    temp = temp.merge(pd.DataFrame(season14[stat]), left_index = True, right_index = True)
    temp = temp.dropna()
    return temp

In [16]:
G, A, PN, SH, FO_W, HIT = [get_stats(x) for x in categories]
frames = [G, A, PN, SH, FO_W, HIT]
for i, df in enumerate(frames):
    frames[i] = df[df.columns[df.corr()[categories[i]] > .5]]
G, A, PN, SH, FO_W, HIT = frames[:]
frames = [G, A, PN, SH, FO_W, HIT]

In [17]:
data = pd.DataFrame()

In [18]:
for i, df in enumerate(frames):
    train = df[0:450]
    train_x = train.drop([categories[i]], 1)
    train_y = train[categories[i]]
    test = df[450:]
    test_x = test.drop([categories[i]], 1)
    test_index = test.index
    regr = linear_model.LinearRegression()
    regr.fit(train_x, train_y)
    print(categories[i], '|', regr.score(test_x, test[categories[i]]))
    data[categories[i]] = regr.predict(test_x).round()
    data[categories[i]][data[categories[i]] < 0] = 0

G | 0.603422548232
A | 0.553061588568
PN | 0.242170162213
SH | 0.607708194172
FO_W | 0.81262377512
HIT | 0.365988007457


In [19]:
data.rename(columns=lambda x: x + '_exp', inplace=True)
data.index = test.index
data.head()

,G_exp,A_exp,PN_exp,SH_exp,FO_W_exp,HIT_exp
Name,,,,,,
Mike.Santorelli,14,15,6,95,211,41
David.Savard,4,10,13,55,7,131
Luca.Sbisa,4,8,11,51,4,99
Marco.Scandella,4,17,10,89,4,87
Mark.Scheifele,16,21,8,111,345,67


In [20]:
season14 = pd.read_csv('Stats/war-on-ice-2014.csv')
season14.index = season14.Name
season14 = season14[categories]
season14.rename(columns=lambda x: x + '_act', inplace=True)
season14 = season14[season14.index.isin(data.index)]

In [22]:
data = data.merge(season14, left_index = True, right_index = True)